In [2]:
# install pyspark lib
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=340bea9c65626655f4a0d24b53cc4a0de144bbd2b1f9ccc6a8055765cb8c884d
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


🌟 𝐏𝐘𝐒𝐏𝐀𝐑𝐊 𝐏𝐫𝐚𝐜𝐭𝐢𝐜𝐞/𝐈𝐧𝐭𝐞𝐫𝐯𝐢𝐞𝐰 𝐏𝐫𝐨𝐛𝐥𝐞𝐦 📊
========================================
Students And Examinations Problem

Write an Pyspark code to find the number of times each student attended each exam.
Order the result table by student_id and subject_name.

𝐒𝐜𝐡𝐞𝐦𝐚 𝐀𝐧𝐝 𝐃𝐚𝐭𝐚:
================
 # Define the schema for the Examinations table
examinations_schema = StructType([
 StructField("student_id", IntegerType(), True),
 StructField("subject_name", StringType(), True)
])

# Data for the Examinations table
examinations_data = [
 (1, "Math"),
 (1, "Physics"),
 (1, "Programming"),
 (2, "Programming"),
 (1, "Physics"),
 (1, "Math"),
 (13, "Math"),
 (13, "Programming"),
 (13, "Physics"),
 (2, "Math"),
 (1, "Math")
]

# Define the schema for the Students table
students_schema = StructType([
 StructField("student_id", IntegerType(), True),
 StructField("student_name", StringType(), True)
])

# Data for the Students table
students_data = [
 (1, "Alice"),
 (2, "Bob"),
 (13, "John"),
 (6, "Alex")
]

# Define the schema for the Subjects table
subjects_schema = StructType([
 StructField("subject_name", StringType(), True)
])

# Data for the Subjects table
subjects_data = [
 ("Math",),
 ("Physics",),
 ("Programming",)
]

In [4]:
# import all necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
# Initialize spark session
spark = SparkSession.builder.master('local').appName('PySpark-day3').getOrCreate()

In [6]:
spark

In [10]:
# define schema and data for examination table
examinations_schema = StructType([
 StructField("student_id", IntegerType(), True),
 StructField("subject_name", StringType(), True)
])

examinations_data = [
 (1, "Math"),
 (1, "Physics"),
 (1, "Programming"),
 (2, "Programming"),
 (1, "Physics"),
 (1, "Math"),
 (13, "Math"),
 (13, "Programming"),
 (13, "Physics"),
 (2, "Math"),
 (1, "Math")
]

In [7]:
# define schema and data for student
students_schema = StructType([
 StructField("student_id", IntegerType(), True),
 StructField("student_name", StringType(), True)
])

students_data = [
 (1, "Alice"),
 (2, "Bob"),
 (13, "John"),
 (6, "Alex")
]

In [8]:
# define schema and data for subject
subjects_schema = StructType([
 StructField("subject_name", StringType(), True)
])

subjects_data = [
 ("Math",),
 ("Physics",),
 ("Programming",)
]

In [11]:
# create df for examination
exam = spark.createDataFrame(schema = examinations_schema, data = examinations_data)
exam.show()

+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+



In [12]:
# create df for student
student = spark.createDataFrame(schema = students_schema, data = students_data)
student.show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|        13|        John|
|         6|        Alex|
+----------+------------+



In [14]:
# create df for subject
subject = spark.createDataFrame(schema = subjects_schema, data = subjects_data)
subject.show()

+------------+
|subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+



In [32]:
# cross join between subject and student to get all possible combination
stu_sub_cross = student.join(subject, how='cross')
# left join exam to above df on both column
result = stu_sub_cross.join(exam, on=['student_id','subject_name'], how='left')
result.show()

+----------+------------+------------+
|student_id|subject_name|student_name|
+----------+------------+------------+
|         6|        Math|        Alex|
|         1| Programming|       Alice|
|        13| Programming|        John|
|         6| Programming|        Alex|
|         2| Programming|         Bob|
|        13|        Math|        John|
|         2|        Math|         Bob|
|         1|        Math|       Alice|
|         1|        Math|       Alice|
|         1|        Math|       Alice|
|         1|     Physics|       Alice|
|         1|     Physics|       Alice|
|        13|     Physics|        John|
|         6|     Physics|        Alex|
|         2|     Physics|         Bob|
+----------+------------+------------+



In [38]:
# group by given 3 col
final_result = result.groupBy(['student_id','subject_name','student_name'])\
                      .count()\
                      .withColumnRenamed('count','attended_exam')\
                      .select(['student_id','student_name','subject_name','attended_exam'])\
                      .orderBy(['student_id','subject_name'])
final_result.show()

+----------+------------+------------+-------------+
|student_id|student_name|subject_name|attended_exam|
+----------+------------+------------+-------------+
|         1|       Alice|        Math|            3|
|         1|       Alice|     Physics|            2|
|         1|       Alice| Programming|            1|
|         2|         Bob|        Math|            1|
|         2|         Bob|     Physics|            1|
|         2|         Bob| Programming|            1|
|         6|        Alex|        Math|            1|
|         6|        Alex|     Physics|            1|
|         6|        Alex| Programming|            1|
|        13|        John|        Math|            1|
|        13|        John|     Physics|            1|
|        13|        John| Programming|            1|
+----------+------------+------------+-------------+

